In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
# Initial setup
n = 100  # Number of samples
np.random.seed(42)

# Generate X matrix with a linearly dependent column (column 4 = 2*column 0 + column 1)
X = np.random.rand(n, 4)  # 4 independent columns
X = np.column_stack([X, 2 * X[:, 0] + X[:, 1]])  # Add linearly dependent column (column 5)

# Generate y using only independent columns (excluding the dependent column)
y = 2 * X[:, 0] + 3 * X[:, 1] - 1.5 * X[:, 2] + np.random.normal(0, 0.1, n)

# Check invertibility of Xt @ X
X = np.column_stack([np.ones(X.shape[0]), X])  # Add intercept column
XtX = X.T @ X
det = np.linalg.det(XtX)

# Round to 4 decimal places
det_rounded = round(det, 4)

print("\n=== Matrix Properties ===")
print(f"{'X shape:':<30}", X.shape)
print(f"{'Determinant of Xt @ X (raw):':<30}", det)
print(f"{'Absolute determinant (rounded):':<30}", np.abs(det_rounded))


=== Matrix Properties ===
X shape:                       (100, 6)
Determinant of Xt @ X (raw):   -7.360311992027218e-08
Absolute determinant (rounded): 0.0


In [3]:
np.linalg.matrix_rank(XtX)

5

# Condition Number: Numerical Stability in Linear Algebra

## Definition
The condition number (κ) is a fundamental metric in numerical linear algebra that quantifies the sensitivity of a matrix to numerical errors during critical operations.

## Mathematical Formulation

### 1. General Definition
For any matrix **A** ∈ ℝⁿˣⁿ with respect to a given norm ‖·‖:


$$
\kappa(A) = 
\begin{cases} 
\|A\| \cdot \|A^{-1}\| & \text{if } A \text{ is invertible} \\ 
\infty & \text{if } A \text{ is singular} 
\end{cases}
$$

| $\kappa(A)$ Range       | Numerical Stability   | Maximum Precision Loss ($\log_{10}\kappa$) |
|-------------------------|----------------------|------------------------------------------|
| $1$–$10^1$              | Excellent            | < 1 decimal digit                       |
| $10^1$–$10^3$           | Good                 | 1–3 decimal digits                     |
| $10^3$–$10^6$           | Acceptable           | 3–6 decimal digits                     |
| $10^6$–$10^{12}$        | Problematic          | 6–12 decimal digits                    |
| $> 10^{12}$             | Unusable             | Complete loss of precision             |



---

Higham, N. J. (2002). "Accuracy and Stability of Numerical Algorithms"

Trefethen, L. N., & Bau, D. (1997). "Numerical Linear Algebra"

In [4]:
# Settings for reproducibility and display
np.random.seed(42)
pd.set_option('display.float_format', '{:.4f}'.format)
n = 100  # Number of samples

# 2. Convert to pandas DataFrames
df_X = pd.DataFrame(X, columns=['Intercept'] + [f'X_{i}' for i in range(1,6)])
df_y = pd.DataFrame(y, columns=['Target'])

# 3. Check matrix properties
XtX = X.T @ X
det_XtX = np.linalg.det(XtX)
cond_num = np.linalg.cond(XtX)

# 4. Enhanced display with color styling
def style_dataframe(df, title, color):
    return (df.head(10)
            .style
            .set_caption(f'<b>{title}</b>')
            .background_gradient(cmap=color, subset=pd.IndexSlice[:, df.columns])
            .set_properties(**{
                'text-align': 'center',
                'font-weight': 'bold',
                'border': '1px solid black'
            })
            .format('{:.4f}')
            .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '14pt'),
                    ('color', 'darkblue'),
                    ('font-weight', 'bold')
                ]
            }]))

# Display tables
display(style_dataframe(df_X, 'Feature Matrix X (First 10 Samples)', 'Blues'))
display(style_dataframe(df_y, 'Target Vector y (First 10 Samples)', 'Reds'))

# Display matrix properties
print(f"\n{'='*50}")
print(f"XtX Matrix Properties:")
print(f"{'Determinant:':<20} {det_XtX:.6f}")
print(f"{'Condition Number:':<20} {cond_num:.2f}")
print(f"{'Invertibility:':<20} {'Invertible' if abs(det_XtX) > 1e-10 else 'Singular'}")
print('='*50)

,Intercept,X_1,X_2,X_3,X_4,X_5
0,1.0000,0.3745,0.9507,0.7320,0.5987,1.6998
1,1.0000,0.1560,0.1560,0.0581,0.8662,0.4680
2,1.0000,0.6011,0.7081,0.0206,0.9699,1.9103
3,1.0000,0.8324,0.2123,0.1818,0.1834,1.8772
4,1.0000,0.3042,0.5248,0.4319,0.2912,1.1332
5,1.0000,0.6119,0.1395,0.2921,0.3664,1.3632
6,1.0000,0.4561,0.7852,0.1997,0.5142,1.6973
7,1.0000,0.5924,0.0465,0.6075,0.1705,1.2313
8,1.0000,0.0651,0.9489,0.9656,0.8084,1.0790
9,1.0000,0.3046,0.0977,0.6842,0.4402,0.7069


,Target
0,2.6338
1,0.6950
2,3.3638
3,1.9981
4,1.5673
5,1.1910
6,2.9779
7,0.4724
8,1.4465
9,0.0851



XtX Matrix Properties:
Determinant:         -0.000000
Condition Number:    23855731764468160.00
Invertibility:       Invertible


In [5]:
X.shape[1]

6

In [6]:
X.shape[0]

100

In [7]:
#(1/det_rounded)*(XtX) 

In [8]:
inv_matrix = np.linalg.inv(XtX) 
inv_matrix

array([[ 1.09795097e-01, -1.24289578e-01, -7.72610888e-02,
        -5.23768168e-02, -5.22623712e-02,  3.52713666e-02],
       [-1.48203394e-01, -2.81474977e+13, -1.40737488e+13,
         1.08082615e-01,  5.72589787e-02,  1.40737488e+13],
       [-8.90493909e-02, -1.40737488e+13, -7.03687442e+12,
         3.98061804e-02,  3.89871548e-03,  7.03687442e+12],
       [-5.23768168e-02,  9.23320837e-02,  3.19309147e-02,
         1.18148999e-01, -5.06724330e-03, -4.41882125e-02],
       [-5.22623712e-02,  5.12840768e-02,  9.11264515e-04,
        -5.06724330e-03,  1.28210192e-01, -2.26545874e-02],
       [ 4.66702949e-02,  1.40737488e+13,  7.03687442e+12,
        -5.20634782e-02, -2.56420384e-02, -7.03687442e+12]])

In [9]:
# Calculate Ridge Regression coefficients
w_ridge = inv_matrix @ X.T @ y

# Print formatted results
print("\n=== Ridge Regression Results ===")
print(f"{'Coefficient':<15} {'Value':<10}")
print("-" * 25)
for i, coef in enumerate(w_ridge):
    print(f"w_{i:<12} {coef:.6f}")
print("\nNote: Coefficients computed with L2 regularization")


=== Ridge Regression Results ===
Coefficient     Value     
-------------------------
w_0            0.178112
w_1            -1.213767
w_2            1.343718
w_3            -1.485282
w_4            -0.024401
w_5            1.499161

Note: Coefficients computed with L2 regularization


In [10]:
y_pred_ridge = X @ w_ridge

In [38]:
lambda_ = 0.1
sse_ridge = np.sum((y - y_pred_ridge)**2)

l2_penalty = lambda_ * np.sum(w_ridge[1:]**2) 
total_loss_ridge =  sse_ridge + l2_penalty

In [39]:
y[10]

1.5774226802005704

In [40]:
y_pred_ridge[10]

1.3753509368923487

In [41]:
print("Ridge SSE:", sse_ridge)

Ridge SSE: 7.626280446609328


In [42]:
# ============================================================================================================================= #
# ============================================================================================================================= #
# ============================================================================================================================= #
# ============================================================================================================================= #

In [70]:
# Set ridge parameter
s = 4  # Regularization strength (lambda)

# Create identity matrix with appropriate dimensions (number of features + 1 for intercept)
I = np.identity(X.shape[1])
I[0, 0] = 0  # No penalty applied to intercept term

In [71]:
I

array([[0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [72]:
s * I

array([[0., 0., 0., 0., 0., 0.],
       [0., 4., 0., 0., 0., 0.],
       [0., 0., 4., 0., 0., 0.],
       [0., 0., 0., 4., 0., 0.],
       [0., 0., 0., 0., 4., 0.],
       [0., 0., 0., 0., 0., 4.]])

In [73]:
det = np.linalg.det(XtX + s * I )
det_rounded = round(det, 4)
det_rounded 

44453411.4858

In [74]:
np.linalg.matrix_rank(XtX+ s * I)

6

In [75]:
np.linalg.cond(XtX+ s * I)

115.42959713945147

In [76]:
np.log10(92.50456818351199)

1.9661631801758332

In [77]:
inv_matrix = np.linalg.inv(XtX + lambda_ * I ) 
inv_matrix

array([[ 1.09158844e-01, -4.14489278e-03, -1.70265329e-02,
        -5.18100939e-02, -5.16608251e-02, -2.53163185e-02],
       [-4.14489278e-03,  6.69346695e+00,  3.28486824e+00,
         5.27653391e-03,  9.00523363e-03, -3.32819787e+00],
       [-1.70265329e-02,  3.28486824e+00,  1.76794318e+00,
        -1.12664865e-02, -1.96161140e-02, -1.66232035e+00],
       [-5.18100939e-02,  5.27653391e-03, -1.12664865e-02,
         1.16751179e-01, -4.97198646e-03, -7.13418676e-04],
       [-5.16608251e-02,  9.00523363e-03, -1.96161140e-02,
        -4.97198646e-03,  1.26537303e-01, -1.60564671e-03],
       [-2.53163185e-02, -3.32819787e+00, -1.66232035e+00,
        -7.13418676e-04, -1.60564671e-03,  1.68128391e+00]])

In [78]:
# Calculate Ridge Regression coefficients
w_ridge = inv_matrix @ X.T @ y

# Display results with formatting
print("\n=== Ridge Regression Coefficients ===")
print(f"{'Coefficient':<12}{'Value':>10}")
print("-" * 22)
for i, coef in enumerate(w_ridge):
    print(f"θ_{i:<9}{coef:>10.4f}")  # θ for coefficients, 4 decimal places
print(f"\nRegularization strength (λ): {s:.1f}")


=== Ridge Regression Coefficients ===
Coefficient      Value
----------------------
θ_0            0.0144
θ_1           -0.2888
θ_2            1.7465
θ_3           -1.4657
θ_4           -0.0209
θ_5            1.1688

Regularization strength (λ): 4.0


In [79]:
y_pred_ridge = X @ w_ridge

In [80]:
sse_ridge = np.sum((y - y_pred_ridge)**2)

l2_penalty = lambda_ * np.sum(w_ridge[1:]**2) 
total_loss_ridge =  sse_ridge + l2_penalty

In [81]:
y[10]

1.5774226802005704

In [82]:
y_pred_ridge[10]

1.6385922687482715

In [56]:
print("Ridge SSE:", sse_ridge)

Ridge SSE: 0.7727727820586238


In [54]:
# Finish